# CFM Data Pre-Processing

In [5]:
# Import Libraries
import pandas as pd
import numpy as np
import MeCab

from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import fasttext
import string
import pickle

import sys
import re



pd.set_option('max_colwidth', 800)

## 1. Load and Process Data

In [2]:
filename_list = ['cfm_train_ja', 'cfm_test_ja', 'cfm_val_ja']

for fn in filename_list:
    if fn.find('train') > 0:
        train_df = pd.read_csv(f'{fn}.csv', encoding = 'utf-8')
    elif fn.find('test') > 0:
        test_df = pd.read_csv(f'{fn}.csv', encoding = 'utf-8')
    elif fn.find('val') > 0:
        val_df = pd.read_csv(f'{fn}.csv', encoding = 'utf-8')
    else:
        pass

In [7]:
def lange_detect(series):
    pretrained_model = "lid.176.bin" 
    model = fasttext.load_model(pretrained_model)
    langs = []
    for cm in series:
        lang = model.predict(cm)[0]
        langs.append(str(lang)[11:13])
    return langs


In [8]:

# Japanese Comments df
train_jp_df = train_df.loc[~train_df['comments_ja'].isna()].drop(columns=['comments_en', 'totalwords_en'])
test_jp_df = test_df.loc[~test_df['comments_ja'].isna()].drop(columns=['comments_en'])
test_jp_df['totalwords_ja'] = test_jp_df['comments_ja'].str.len()
val_jp_df = val_df.loc[~val_df['comments_ja'].isna()].drop(columns=['comments_en'])
val_jp_df['totalwords_ja'] = val_jp_df['comments_ja'].str.len()


## Combined df for Japanese comments 
jp_dfs = [train_jp_df, test_jp_df, val_jp_df]
## modify lang and rename comment col
for j in jp_dfs:
    j['lang'] = 'jp'
    j.rename(columns = {'comments_ja': 'comments', 'totalwords_ja': 'init_totalwords'}, inplace =  True)
all_jp_dfs = pd.concat(jp_dfs, sort = False)
# remove punctuation
all_jp_dfs['comments_no_punc'] = all_jp_dfs['comments'].str.replace('[^\w\s]','')
all_jp_dfs['comments_no_punc']=all_jp_dfs['comments_no_punc'].apply(lambda x: re.sub('\w*\d\w*','', x))

# detect language
all_jp_dfs['lang_detect'] = lange_detect(all_jp_dfs['comments_no_punc'])
print(f'The size of the observations for raw dataset of Japanese comments is {all_jp_dfs.shape[0]}')
# Filter out the Japanese comments
all_jp_dfs = all_jp_dfs[all_jp_dfs['lang_detect'].isin(['ja'])] 
print(f'The size of the observations for the cleaned dataset of Japanese comments is {all_jp_dfs.shape[0]}')


The size of the observations for raw dataset of Japanese comments is 76115
The size of the observations for the cleaned dataset of Japanese comments is 50544


In [10]:
stop_words_jp = [ 'てる', 'いる', 'なる', 'れる', 'する', 'ある', 'こと', 'これ', 'さん', 'して', \

'くれる', 'やる', 'くださる', 'そう', 'せる', 'した', '思う', \

'それ', 'ここ', 'ちゃん', 'くん', '', 'て','に','を','は','の', 'が', 'と', 'た', 'し', 'で', \

'ない', 'も', 'な', 'い', 'か', 'ので', 'よう', '', 'れ','さ','なっ', 'br', '年', '月', '日']

def mecab_parser(text):
    mct = MeCab.Tagger()
    mct.parse('')
    node = mct.parseToNode(text)
    output = []
    
    while node:
        if node.surface != "":
            word_type = node.feature.split(",")[0]
            if word_type in ["形容詞", "動詞","名詞", "副詞"]:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return " ".join(output)

all_jp_dfs['tokenized'] = all_jp_dfs['comments_no_punc'].apply(mecab_parser)
all_jp_dfs.head()

,comments,lang,label,init_totalwords,comments_no_punc,lang_detect,tokenized
0,最悪の商品です。<br />購入後、一週間たたずに発火しました。,jp,1,32,最悪の商品ですbr 購入後一週間たたずに発火しました,ja,最悪 商品 購入 一 週間 たた 発火 し
1,使用中に発火した。.,jp,1,10,使用中に発火した,ja,使用 発火 し
2,USBコネクター部分が発火しました。.,jp,1,19,USBコネクター部分が発火しました,ja,USB コネクター 部分 発火 し
3,ブレーカーを落として、電球とこちらを交換して、スイッチを入れたら爆発しましたけど、、、、、、、､めちゃくちゃ怖かったです。。。その後、冷えてから外して、元々点けていた電球を取り付けたら普通でした。こんな経験はじめてでした。.,jp,1,112,ブレーカーを落として電球とこちらを交換してスイッチを入れたら爆発しましたけどめちゃくちゃ怖かったですその後冷えてから外して元々点けていた電球を取り付けたら普通でしたこんな経験はじめてでした,ja,ブレーカー 落とし 電球 交換 し スイッチ 入れ 爆発 し めちゃくちゃ 怖かっ 後 冷え 外し 元々 点け い 電球 取り付け 普通 経験 はじめて
5,車載の充電器として使用していましたが、運転中に発火しました。たまたま止まっていたので問題はなかったですが、動いている時であれば事故につながっていました。<br />返金していただきたいです。,jp,1,95,車載の充電器として使用していましたが運転中に発火しましたたまたま止まっていたので問題はなかったですが動いている時であれば事故につながっていましたbr 返金していただきたいです,ja,車載 充電 し 使用 し い 運転 発火 し たまたま 止まっ い 問題 なかっ 動い いる 時 あれ 事故 つながっ い br 返金 し いただき


In [11]:
all_jp_dfs = all_jp_dfs[['tokenized', 'label', 'init_totalwords']]

In [14]:

with open('./clean_data/jp_comments_clean.pkl', 'wb') as pickle_file:
    pickle.dump(all_jp_dfs, pickle_file)